In [2]:
import os           
import re           
import pandas as pd 
import numpy as np  
import cv2 as cv 
from sklearn.model_selection import train_test_split   

In [2]:
def create_database(directory, n_samples):
    check_if_exists = os.path.exists('images')
    if check_if_exists and len(os.listdir('images')) > 0:
        print('Images already exist in the directory')
        return
    pattern_1 = r'Y_(\d+)p(\d+)dB'
    pattern_2 = r'Y_(\d+)dB'
    song_pattern = r'Song(\d+)'
    
    for dirs in os.listdir(directory)[::-1]:
        for files in os.listdir(os.path.join(directory, dirs)):
            name_match = re.search(song_pattern, files)
            if name_match:
                name = name_match.group(0)
            try:
                match = re.search(pattern_1, files)
                dB = f'{match.group(1)}.{match.group(2)}'
            except:
                match = re.search(pattern_2, files)
                dB = f'{match.group(1)}'

            path = os.path.join(directory, dirs, files)
            data = pd.read_csv(path)
            data = data.to_numpy()
            
            for i in range(0, data.shape[0], n_samples):
                data_i = data[i:i + n_samples]
                x, y = data_i[:, 0], data_i[:, 1]
                hist, _, _ = np.histogram2d(x, y, bins=32, range=[[-5, 5], [-5, 5]])
                hist = cv.normalize(hist, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)
                
                if not os.path.exists('images'):
                    os.makedirs('images')
                    
                cv.imwrite(f'images/{dirs}_{dB}dB_sample({i//n_samples})_{name}.png', hist)
                
    print('Images have been created successfully')
    return

# Llamada a la función
img_db_created = create_database('0km_0dBm_consY', 13177)

Images have been created successfully


In [3]:
images = []
labels = []
sp = []

pattern_1 = r'(\d+).(\d+)dB'
pattern_2 = r'(\d+)dB'
sp_pattern1 = r'(\d+p\d+)GHz'
sp_pattern2 = r'(\d+)GHz'

for filename in os.listdir('images'):
    if filename.endswith('.png'):

        try:
            match_sp = re.search(sp_pattern1, filename)
            sp_value = f'{match_sp.group(0)}'
        except:
            match_sp = re.search(sp_pattern2, filename)
            sp_value = f'{match_sp.group(0)}'

        try:
            match_label = re.search(pattern_1,filename)
            label = f'{match_label.group(1)}.{match_label.group(2)}'
        except:
            match_label = re.search(pattern_2,filename)
            label = f'{match_label.group(1)}'
            
        img_path = os.path.join('images', filename)
        img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        img = img / 255.0
        
        images.append(img)
        labels.append(label)
        sp.append(sp_value)

images = np.array(images)
labels = np.array(labels, dtype=np.float32)
sp = np.array(sp)

In [70]:
print(images.shape)
print(labels[1])
print(sp[1])

(2157, 32, 32)
26.07
28p5GHz


In [4]:
images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])
labels = labels.reshape(labels.shape[0], 1)
min, max = np.min(labels), np.max(labels)
labels = (labels - min) / (max - min)
sps = ["28p5GHz", "29GHz", "29p5GHz", "30GHz", "30p5GHz", "31GHz", "31p5GHz", "32GHz", "32p5GHz", "33GHz", "33p5GHz", "34GHz", "34p5GHz", "35GHz", "35p5GHz", "36GHz", "37p5GHz", "50GHz"]
sp_2_id = {sp: idx for idx, sp in enumerate(sps)}
sp_map = np.array([sp_2_id[valor] for valor in sp])
sp_ohe = np.eye(len(np.unique(sp_map)))[sp_map]

In [72]:
print(images.shape)
print(labels.shape)
print(sp_ohe.shape)

(2157, 1024)
(2157, 1)
(2157, 18)


In [5]:
images_train, images_test, labels_train, labels_test, sp_train, sp_test = train_test_split(
    images, labels, sp_ohe, test_size=0.2, random_state=42
)

images_test, images_val, labels_test, labels_val, sp_est, sp_val = train_test_split(
    images_test, labels_test, sp_test, test_size=0.5, random_state=42
) 

In [6]:
print(len(images_train))
print(len(images_val))
print(len(images_test))

print(len(labels_train))
print(len(labels_val))
print(len(labels_test))

1725
216
216
1725
216
216


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model

input1 = Input((1024,))
input2 = Input((18,))
x = Dense(512,activation = 'relu')(input1)
x = Dense(256,activation = 'relu')(x)
x = Dense(128,activation = 'relu')(x)
x = Dense(64,activation = 'relu')(x)
x = Dense(32,activation = 'relu')(x)
x = concatenate([x,input2])
output = Dense(1)(x)
model = Model([input1, input2], output)
model.compile(
        optimizer='adam',
        loss='mae'
    )
model.fit([images_train, sp_train], labels_train, epochs=200, validation_data=([images_val, sp_val],labels_val))

Epoch 1/200


c:\Users\Paula Hios Torres\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1']. Received: the structure of inputs=('*', '*')
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.2983 - val_loss: 0.2440
Epoch 2/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.2306 - val_loss: 0.2156
Epoch 3/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1909 - val_loss: 0.1935
Epoch 4/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1757 - val_loss: 0.1832
Epoch 5/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1479 - val_loss: 0.1617
Epoch 6/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1282 - val_loss: 0.1381
Epoch 7/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1104 - val_loss: 0.1373
Epoch 8/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1103 - val_loss: 0.1213
Epoch 9/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1031 - val_loss: 0.1258
Epoch 10/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0937 - val_loss: 0.1104
Epoch 11/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0845 - val_loss: 0.1114
Epoch 12/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.08

In [8]:
from sklearn.metrics import mean_absolute_error

preds = model.predict([images_test, sp_est])
preds_n = preds*(max-min) + min
labels_n = labels_test*(max-min) + min
mae = mean_absolute_error(labels_n, preds_n)
print(mae)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1.2681684


In [76]:
import tensorflow as tf
import keras_tuner as kt
import numpy as np

# Semilla generador aleatorio
tf.random.set_seed(123)

def crear_modelo(hp):
    # Contenedor vacío
    modelo = tf.keras.Sequential()

    # Capa de entrada: cada dato tendrá 1024 elementos
    modelo.add(tf.keras.layers.Dense(1024, activation="relu", input_shape=(1024,)))

    # Hiperparámetro: número de capas ocultas (entre 3 y 8)
    num_layers = hp.Int("num_layers", min_value=3, max_value=5, step=1)

    for i in range(num_layers):
        # Hiperparámetro: número de neuronas por capa (64 a 512, con paso de 64)
        units = hp.Int(f"units_{i}", min_value=64, max_value=256, step=64)

        # Hiperparámetro: función de activación para la capa (ReLU, Sigmoid, Tanh, etc.)
        activation = hp.Choice(f"activation_{i}", values=["relu", "sigmoid", "tanh", "elu"])

        # Agregar la capa con los hiperparámetros seleccionados
        modelo.add(tf.keras.layers.Dense(units=units, activation=activation))

    # Capa de salida
    modelo.add(tf.keras.layers.Dense(units=1, activation="linear"))

    # Hiperparámetro: tasa de aprendizaje del optimizador Adam
    hp_lr = hp.Choice("learning_rate", values=[0.1, 0.01, 0.001])

    # Compilación del modelo usando Adam
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
        loss="mean_absolute_error",
        metrics=["mae"]
    )

    return modelo

In [77]:
# Configurar el tuner
tuner = kt.RandomSearch(
    hypermodel=crear_modelo,
    objective="val_mae",
    max_trials=50,
    directory="afinacion_red",
    project_name="trials",
    overwrite=True
)

In [79]:
# Ejecutar la búsqueda
tuner.search(
    images_train, labels_train,
    epochs=50,  
    validation_data=(images_val, labels_val)
)

Trial 45 Complete [00h 00m 18s]
val_mae: 0.16436663269996643

Best val_mae So Far: 0.15161283314228058
Total elapsed time: 00h 12m 14s

Search: Running Trial #46

Value             |Best Value So Far |Hyperparameter
4                 |3                 |num_layers
256               |128               |units_0
elu               |relu              |activation_0
256               |256               |units_1
relu              |tanh              |activation_1
64                |192               |units_2
sigmoid           |tanh              |activation_2
0.001             |0.001             |learning_rate
128               |128               |units_3
tanh              |tanh              |activation_3
64                |256               |units_4
tanh              |relu              |activation_4

Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.2459 - mae: 0.2459 - val_loss: 0.1671 - val_mae: 0.1671
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.1570 - mae: 0.1570 

KeyboardInterrupt: 